In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'lrp'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
# First, we get the 800 timepoints we are considering as a list, to use for indices given by cluster-based permutation later
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')
pp = 0 #only need one participant

# we can use aligned data, any movement condition (e.g. leftmoves)

id_directory = os.path.join(data_directory, 'p%03d/' % pp)
pp_directory = os.path.join(id_directory, erps)
fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (pp, 'leftmoves', 'SL', 'aligned'))
evoked = mne.read_evokeds(fname)
df = evoked[0].to_data_frame()
time = df['time'].tolist()
time = time[200:401] #get only timepoints we want

In [ ]:
# Load data for each condition
# Create separate lists for C3 and C4, along with right and left movements, so that we may calculate for LRPs
# transform data into array accepted for cluster-based permutation test in mne
# output: (n_participants, n_timepts) for each condition

root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

# #specify channels we need - LRP
channels = ['C3', 'C4']

#ALIGNED
#read in evoked object
movenames = ['rightmoves', 'leftmoves']

for channel in range(0, len(channels)):
    for move in range(0, len(movenames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], movenames[move], 'SL', 'aligned'))
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
            evoked = evoked[0] #get first list then index timepts below
            evoked = evoked[200:401] #get timepts we are interested in
            evoked_list.append(evoked)
        if move == 0 and channel == 0:
            aln_right_C3 = evoked_list
        elif move == 0 and channel == 1:
            aln_right_C4 = evoked_list
        elif move == 1 and channel == 0:
            aln_left_C3 = evoked_list
        elif move == 1 and channel == 1:
            aln_left_C4 = evoked_list
            
#ALIGNED FOR ROTATION
movenames = ['rightmoves', 'leftmoves']

for channel in range(0, len(channels)):
    for move in range(0, len(movenames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], movenames[move], 'SL', 'aln_rot'))
            if os.path.isfile(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
            evoked = evoked[0] #get first list then index timepts below
            evoked = evoked[200:401] #get timepts we are interested in
            evoked_list.append(evoked)
        if move == 0 and channel == 0:
            alnrot_right_C3 = evoked_list
        elif move == 0 and channel == 1:
            alnrot_right_C4 = evoked_list
        elif move == 1 and channel == 0:
            alnrot_left_C3 = evoked_list
        elif move == 1 and channel == 1:
            alnrot_left_C4 = evoked_list
            
#ALIGNED FOR MIRROR
movenames = ['rightmoves', 'leftmoves']

for channel in range(0, len(channels)):
    for move in range(0, len(movenames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], movenames[move], 'SL', 'aln_mir'))
            if os.path.isfile(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
            evoked = evoked[0] #get first list then index timepts below
            evoked = evoked[200:401] #get timepts we are interested in
            evoked_list.append(evoked)
        if move == 0 and channel == 0:
            alnmir_right_C3 = evoked_list
        elif move == 0 and channel == 1:
            alnmir_right_C4 = evoked_list
        elif move == 1 and channel == 0:
            alnmir_left_C3 = evoked_list
        elif move == 1 and channel == 1:
            alnmir_left_C4 = evoked_list
            
#PERTURBED CONDITIONS - only half of N in rot and mir due to considering only left and right movements
conditionnames = ['sml', 'lrg']

#ROTATED
for condition in range(0, len(conditionnames)):
    for channel in range(0, len(channels)):
        for move in range(0, len(movenames)):
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], movenames[move], conditionnames[condition], 'rot'))
                if os.path.isfile(fname) == False:
                    continue
                evoked = mne.read_evokeds(fname)
                evoked = evoked[0]
                evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
                evoked = evoked[0] #get first list then index timepts below
                evoked = evoked[200:401] #get timepts we are interested in
                evoked_list.append(evoked)
            if move == 0 and channel == 0 and condition == 0:
                rot_small_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 0:
                rot_small_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 0:
                rot_small_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 0:
                rot_small_left_C4 = evoked_list
            elif move == 0 and channel == 0 and condition == 1:
                rot_large_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 1:
                rot_large_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 1:
                rot_large_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 1:
                rot_large_left_C4 = evoked_list

#MIRROR
for condition in range(0, len(conditionnames)):
    for channel in range(0, len(channels)):
        for move in range(0, len(movenames)):
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], movenames[move], conditionnames[condition], 'mir'))
                if os.path.isfile(fname) == False:
                    continue
                evoked = mne.read_evokeds(fname)
                evoked = evoked[0]
                evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
                evoked = evoked[0] #get first list then index timepts below
                evoked = evoked[200:401] #get timepts we are interested in
                evoked_list.append(evoked)
            if move == 0 and channel == 0 and condition == 0:
                mir_small_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 0:
                mir_small_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 0:
                mir_small_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 0:
                mir_small_left_C4 = evoked_list
            elif move == 0 and channel == 0 and condition == 1:
                mir_large_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 1:
                mir_large_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 1:
                mir_large_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 1:
                mir_large_left_C4 = evoked_list

#RANDOM
for condition in range(0, len(conditionnames)):
    for channel in range(0, len(channels)):
        for move in range(0, len(movenames)):
            evoked_list = []
            for pp in participants:
                id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
                pp_directory = os.path.join(id_directory, erps)
                fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], movenames[move], conditionnames[condition], 'rdm'))
                if os.path.isfile(fname) == False:
                    continue
                evoked = mne.read_evokeds(fname)
                evoked = evoked[0]
                evoked = evoked.get_data(picks=channels[channel]) #will give data of shape (n_channels, n_timepts)
                evoked = evoked[0] #get first list then index timepts below
                evoked = evoked[200:401] #get timepts we are interested in
                evoked_list.append(evoked)
            if move == 0 and channel == 0 and condition == 0:
                rdm_small_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 0:
                rdm_small_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 0:
                rdm_small_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 0:
                rdm_small_left_C4 = evoked_list
            elif move == 0 and channel == 0 and condition == 1:
                rdm_large_right_C3 = evoked_list
            elif move == 1 and channel == 0 and condition == 1:
                rdm_large_left_C3 = evoked_list
            elif move == 0 and channel == 1 and condition == 1:
                rdm_large_right_C4 = evoked_list
            elif move == 1 and channel == 1 and condition == 1:
                rdm_large_left_C4 = evoked_list

In [ ]:
#Next, we calculate LRPs: (right C3 - right C4) - (left C3 - left C4)

perturbations = ['aln', 'alnrot', 'alnmir', 'rot', 'rdm', 'mir']
conditionnames = ['sml', 'lrg']

for p in range(0, len(perturbations)):
    if p == 0:
        rightdiff = np.subtract(aln_right_C3, aln_right_C4)
        leftdiff = np.subtract(aln_left_C3, aln_left_C4)
        aln_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 1:
        rightdiff = np.subtract(alnrot_right_C3, alnrot_right_C4)
        leftdiff = np.subtract(alnrot_left_C3, alnrot_left_C4)
        alnrot_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 2:
        rightdiff = np.subtract(alnmir_right_C3, alnmir_right_C4)
        leftdiff = np.subtract(alnmir_left_C3, alnmir_left_C4)
        alnmir_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 3:
        for condition in range(0, len(conditionnames)):
            if condition == 0:
                rightdiff = np.subtract(rot_small_right_C3, rot_small_right_C4)
                leftdiff = np.subtract(rot_small_left_C3, rot_small_left_C4)
                small_rot_lrp = np.subtract(rightdiff, leftdiff)
            elif condition == 1:
                rightdiff = np.subtract(rot_large_right_C3, rot_large_right_C4)
                leftdiff = np.subtract(rot_large_left_C3, rot_large_left_C4)
                large_rot_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 4:
        for condition in range(0, len(conditionnames)):
            if condition == 0:
                rightdiff = np.subtract(rdm_small_right_C3, rdm_small_right_C4)
                leftdiff = np.subtract(rdm_small_left_C3, rdm_small_left_C4)
                small_rdm_lrp = np.subtract(rightdiff, leftdiff)
            elif condition == 1:
                rightdiff = np.subtract(rdm_large_right_C3, rdm_large_right_C4)
                leftdiff = np.subtract(rdm_large_left_C3, rdm_large_left_C4)
                large_rdm_lrp = np.subtract(rightdiff, leftdiff)
    elif p == 5:
        for condition in range(0, len(conditionnames)):
            if condition == 0:
                rightdiff = np.subtract(mir_small_right_C3, mir_small_right_C4)
                leftdiff = np.subtract(mir_small_left_C3, mir_small_left_C4)
                small_mir_lrp = np.subtract(rightdiff, leftdiff)
            elif condition == 1:
                rightdiff = np.subtract(mir_large_right_C3, mir_large_right_C4)
                leftdiff = np.subtract(mir_large_left_C3, mir_large_left_C4)
                large_mir_lrp = np.subtract(rightdiff, leftdiff)

In [ ]:
# Then we can run cluster based permutation tests, comparing our conditions
# Note, rot and mir conditions only have 16 participants, while aligned and random have 32
# Hence we grab corresponding aln participants for rot and mir above

# Comparing two ERP signals is just the same as taking their difference (ERP1 minus ERP2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
    np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# Comparing participants between rotation and mirror would be an independent samples comparison
# permutation function should be modified
# F dist used here, but oneway F test = independent t test when dealing with two independent conditions only
def get_ind_clust_perm_test(conditionA, conditionB, pval, n_permutations, n_conditions = 2):
    #define cluster forming threshold based on p-value
    dfn = n_conditions - 1  # degrees of freedom numerator
    dfd = len(participants) - n_conditions  # degrees of freedom denominator
    thresh = scipy.stats.f.ppf(1 - pval, dfn=dfn, dfd=dfd)  
    #run cluster-based permutation test
    F_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_test([conditionA, conditionB], threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return F_0, clust_idx, clust_pvals, H0

In [ ]:
# First, we can compare each condition to aligned. Note that rot and mir have different aln conditions
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in ERP plots

# RANDOM
flists = [small_rdm_lrp, large_rdm_lrp]
conditionnames = ['smallrdm', 'largerdm']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], aln_lrp, p, perms)
#     print(clust_idx, clust_pvals)
    if len(clust_idx) == 0:
        condition.append(conditionnames[c])
        clust_idx_start.append(np.nan)
        clust_idx_end.append(np.nan)
        time_start.append(np.nan)
        time_end.append(np.nan)
        p_values.append(np.nan)
    else:
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[f])

# ROTATION
participants = [0,1,2,3,8,9,10,11,16,17,18,19,24,25,26,27] #need to specify pp IDs we need
flists = [small_rot_lrp, large_rot_lrp]
conditionnames = ['smallrot', 'largerot']

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], alnrot_lrp, p, perms)
#     print(clust_idx, clust_pvals)
    if len(clust_idx) == 0:
        condition.append(conditionnames[c])
        clust_idx_start.append(np.nan)
        clust_idx_end.append(np.nan)
        time_start.append(np.nan)
        time_end.append(np.nan)
        p_values.append(np.nan)
    else:
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[f])

# MIRROR            
participants = [4,5,6,7,12,13,14,15,20,21,22,23,28,29,30,31] #need to specify pp IDs we need
flists = [small_mir_lrp, large_mir_lrp]
conditionnames = ['smallmir', 'largemir']

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], alnmir_lrp, p, perms)
#     print(clust_idx, clust_pvals)
    if len(clust_idx) == 0:
        condition.append(conditionnames[c])
        clust_idx_start.append(np.nan)
        clust_idx_end.append(np.nan)
        time_start.append(np.nan)
        time_end.append(np.nan)
        p_values.append(np.nan)
    else:
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_vsAligned_SmallLarge_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next, we subtract aligned from each condition, so that we can compare small vs large in each perturbation type
diffconds = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']
smallrot_diff = []
largerot_diff = []
smallrdm_diff = []
largerdm_diff = []
smallmir_diff = []
largemir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(small_rot_lrp, alnrot_lrp)
        smallrot_diff.append(diffevks)
        smallrot_diff = smallrot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(large_rot_lrp, alnrot_lrp)
        largerot_diff.append(diffevks)
        largerot_diff = largerot_diff[0]
    elif cond == 2:
        diffevks = np.subtract(small_rdm_lrp, aln_lrp)
        smallrdm_diff.append(diffevks)
        smallrdm_diff = smallrdm_diff[0]
    elif cond == 3:
        diffevks = np.subtract(large_rdm_lrp, aln_lrp)
        largerdm_diff.append(diffevks)
        largerdm_diff = largerdm_diff[0]
    elif cond == 4:
        diffevks = np.subtract(small_mir_lrp, alnmir_lrp)
        smallmir_diff.append(diffevks)
        smallmir_diff = smallmir_diff[0]
    elif cond == 5:
        diffevks = np.subtract(large_mir_lrp, alnmir_lrp)
        largemir_diff.append(diffevks)
        largemir_diff =largemir_diff[0]

In [ ]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of SMALL vs LARGE comparisons
# This information can then be included in ERP difference wave plots

conds = ['rot_diff', 'rdm_diff', 'mir_diff']
conditionnames = ['rot', 'rdm', 'mir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        participants = [0,1,2,3,8,9,10,11,16,17,18,19,24,25,26,27]
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(largerot_diff, smallrot_diff, p, perms)
#         print(clust_idx, clust_pvals)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 1:
        participants = list(range(0,32))
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(largerdm_diff, smallrdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:    
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 2:
        participants = [4,5,6,7,12,13,14,15,20,21,22,23,28,29,30,31]
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(largemir_diff, smallmir_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:    
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_SmallvsLarge_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next step is to subtract small from large condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
rot_diff = []
rdm_diff = []
mir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(largerot_diff, smallrot_diff)
        rot_diff.append(diffevks)
        rot_diff = rot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(largerdm_diff, smallrdm_diff)
        rdm_diff.append(diffevks)
        rdm_diff = rdm_diff[0]
    elif cond == 2:
        diffevks = np.subtract(largemir_diff, smallmir_diff)
        mir_diff.append(diffevks)
        mir_diff = mir_diff[0]

In [ ]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of PERTURBATION comparisons
# This information can then be included in ERP difference wave plots

conds = ['rotvmir', 'rotvrdm', 'mirvrdm']
conditionnames = ['rotvmir', 'rotvrdm', 'mirvrdm']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        participants = list(range(0,32)) # rot plus mir participants gives N = 32
        F_0, clust_idx, clust_pvals, H0 = get_ind_clust_perm_test(rot_diff, mir_diff, p, perms)
#         print(clust_idx, clust_pvals)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 1:
        #random has 32 N but rotation has 16, get corresponding participants in rdm
        participants = [0,1,2,3,8,9,10,11,16,17,18,19,24,25,26,27]
        rdm_subdat = [v for i,v in enumerate(rdm_diff) if i in participants]
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, rdm_subdat, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 2:
        #random has 32 N but mirror has 16, get corresponding participants in rdm
        participants = [4,5,6,7,12,13,14,15,20,21,22,23,28,29,30,31]
        rdm_subdat = [v for i,v in enumerate(rdm_diff) if i in participants]
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(mir_diff, rdm_subdat, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_SmallLarge_PerturbTypeComp_%s.csv' % (erps))
perm_test.to_csv(perm_test_filename)